# Terminologia adotada em modelos de redes

Uma rede é formada por um conjunto de pontos e de retas conectando certos pares de pontos. 

Os pontos são chamados **nós** (ou vértices). As retas são chamadas **arcos** (ou ligações ou bordas ou ramificações).

Os arcos de uma rede podem ter um fluxo de algum tipo através deles. Se o fluxo através de um arco for permitido apenas em uma direção (por exemplo, uma via de mão única), o arco é dito **arco direcionado**.

Se o fluxo através de um arco for permitido em ambas as direções (por exemplo, uma tubulação que pode ser usada para bombear fluido em ambas as direções), o arco é chamado **arco não-direcionado**.

Um **caminho** entre dois nós é uma seqüência de arcos distintos conectando esses nós. Um caminho que começa e termina no mesmo nó é chamado **ciclo**.

Uma **rede conectada** é uma rede na qual todo par de nós é conectado.

# Problema do Caminho Mais Curto

Considere uma rede conectada e não-direcionada com dois nós especiais chamados origem e destino. Associado a cada uma das ligações (arcos não-direcionados) está uma distância não-negativa. 

<img src="./img/seervarda-park.png">

O objetivo é encontrar o caminho mais curto (o caminho com a distância total mínima) da origem ao destino.

- Grafos não direcionados: algoritmo de Dijkstra com complexidade de tempo $O(V^2)$
- Grafos direcionados com pesos e sem ciclos: Algoritmo Bellman-Ford Complexidade de tempo $O(VA)$


In [1]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"
#include "ortools/graph/shortestpaths.h"

using namespace operations_research;

In [2]:
const int64 M = 100;

const std::vector<char> nodes{'O','A','B','C','D','E','T'};

const std::vector<std::vector<int64>> arcs{
/*    O, A, B, C, D, E, T*/
/*O*/{M, 2, 5, 4, M, M, M}, 
/*A*/{2, M, 2, M, 7, M, M}, 
/*B*/{5, 2, M, 1, 4, 3, M}, 
/*C*/{4, M, 1, M, M, 4, M}, 
/*D*/{M, 7, 4, M, M, 1, 5}, 
/*E*/{M, M, 3, 4, 1, M, 7}, 
/*T*/{M, M, M, M, 5, 7, M}  
}

In [3]:
{
    int nos = nodes.size();
    int origem = 0;
    int destino = nos-1;
    std::vector<int> paths;
    bool found = DijkstraShortestPath(nos, origem, destino,
                                     [](int x, int y) { return arcs[x][y]; },
                                      M, &paths);
    if (found){
        int i = paths.size()-1;
        std::cout << nodes[paths[i]];
        while(--i >= 0)
            std::cout << "->" << nodes[paths[i]];
    }
}

O->A->B->D->T

# Árvore Geradora Mínima

Uma "árvore" pode ser "cultivada" adicionando-se um arco (ou "ramificação") por vez a partir da rede original em uma certa direção.

O primeiro arco pode ir para qualquer lugar para conectar algum par de nós. A partir daí, cada arco novo deve estar entre um nó que já esteja conectado a outros nós e um novo nó não previamente conectado a qualquer outro nó. 

Adicionar um nó dessa maneira impede a criação de um ciclo e garante que o número de nós conectados seja uma unidade a mais que o número de arcos. 

Cada arco novo cria uma **árvore** maior que é uma rede conectada (de algum subconjunto dos $n$ nós) que não contém nenhum ciclo não-direcionado. 

Assim que o arco $(n - 1)$ tiver sido adicionado, o processo pára, pois a árvore resultante abrange (conecta) todos os $n$ nós. 

Essa árvore é chamada **árvore de expansão**, isto é, uma rede conectada de todos os n nós que não contêm nenhum ciclo não-direcionado. 

Cada uma das árvores de expansão tem exatamente $n - 1$ arcos, visto que este é o número mínimo de arcos necessário para ter uma rede conectada e o número máximo possível sem ter ciclos não-direcionados.

In [4]:
#include "ortools/graph/graph.h"
#include "ortools/graph/minimum_spanning_tree.h"

using namespace operations_research;

## Algoritmo "Guloso" de Prim

1. Selecione qualquer nó arbitrariamente e depois o conecte ao nó distinto mais próximo.
2. Identifique o nó desconectado que esteja mais próximo a qualquer um dos nós já conectados. 
3. Repita a etapa anterior até que todos os nós tenham sido conectados.

<img src="./img/seervarda-park-mst.png">

### Complexidade de Tempo: $O(V^2)$

In [5]:
{
    util::ReverseArcListGraph<> graph;
    
    for (int tail=0; tail<nodes.size(); tail++) 
        for (int head=tail+1; head<nodes.size(); head++)
            if (arcs[tail][head] < M)
                graph.AddArc(tail, head); 
    
    const auto arc_cost = [&graph](int arc) -> int64 {
                           return arcs[graph.Tail(arc)][graph.Head(arc)];
                        };
    
    std::vector<int> MST = BuildPrimMinimumSpanningTree(graph, arc_cost);
    
    for(auto arc : MST){
        auto tail = graph.Tail(arc);
        auto head = graph.Head(arc);
        std::cout << nodes[tail] << "->" << nodes[head] << std::endl;
    }
}

O->A
A->B
B->C
B->E
B->D
D->T


## Algoritmo de Kruskal

1. crie uma floresta F (um conjunto de árvores), onde cada vértice no grafo é uma árvore separada
2. crie um conjunto S contendo todas os arcos do grafo
3. enquanto S não está vazio e F ainda não está completo
    - remova um arco com peso mínimo de S
    - se o arco removido conecta duas árvores diferentes, adicione-a à floresta F, combinando duas árvores em uma única árvore

No término do algoritmo, a floresta forma uma floresta de expansão mínima do grafo. 

Se o grafo for conectado, a floresta tem um único componente e forma uma árvore geradora mínima.

### Complexidade de Tempo: $O(A log V)$

In [6]:
{
    util::ReverseArcListGraph<> graph;
    
    for (int tail=0; tail<nodes.size(); tail++) 
        for (int head=tail+1; head<nodes.size(); head++)
            if (arcs[tail][head] < M)
                graph.AddArc(tail, head); 
    
    const auto arc_cost = [&graph](int arc) -> int64 {
                           return arcs[graph.Tail(arc)][graph.Head(arc)];
                        };
    
    std::vector<int> MST = BuildKruskalMinimumSpanningTree(graph, 
                                                          [&arc_cost](int a, int b) { return arc_cost(a) < arc_cost(b); });
    
    for(auto arc : MST){
        auto tail = graph.Tail(arc);
        auto head = graph.Head(arc);
        std::cout << nodes[tail] << "->" << nodes[head] << std::endl;
    }
}

B->C
D->E
O->A
A->B
B->E
D->T
